In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
# Read in XLSX files
folder_path = "Input"

files = os.listdir(folder_path)

matching_claim = [file for file in files if "Incline Redpoint_Claim_MRL_Record @ " in file]

if not matching_claim:
    print("No matching claim file found.")
else:
    # Use the first matching file found (you may need to adjust this logic if multiple matches are possible)
    claim_file_path = os.path.join(folder_path, matching_claim[0])
    claim_df = pd.read_excel(claim_file_path, sheet_name="Premium_And_Loss_by_Coverage_Cl", skiprows=1)

matching_premium = [file for file in files if "Premium_And_Loss_by_Coverage_Written_Earned Premium_Details_INBG @ " in file]

if not matching_premium:
    print("No matching premium file found.")
else:
    # Use the first matching file found (you may need to adjust this logic if multiple matches are possible)
    premium_file_path = os.path.join(folder_path, matching_premium[0])

    # Step 6: Read in premium xlsx file as a DataFrame
    premium_df = pd.read_excel(premium_file_path, sheet_name="WP detail", skiprows = 7)


### Check: whether reported claim has associated policy number in the Premium BDX

In [2]:
claim_df

NameError: name 'claim_df' is not defined

In [15]:
#Check if Claims policy is in the Premium BDX
is_every_inb2_policy_included = all(
    claim_df[claim_df['Program Name'] == 'INB2']['Policy Number'].isin(claim_df['Policy Number'].unique())
)

if is_every_inb2_policy_included:
    print("Every Policy Number in Claim BDX is included in the Premium Policy Numbers.")
else:
    print("Not every Policy Number in Claim BDX is included in the Premium Policy Numbers.")


Every Policy Number in Claim BDX is included in the Premium Policy Numbers.
